In [1]:
pip install opendatasets

In [2]:
import opendatasets as od
od.download(
    "https://www.kaggle.com/datasets/kazanova/sentiment140")

Skipping, found downloaded files in "./sentiment140" (use force=True to force download)


# Import Data to Colab

In [3]:
import pandas as pd
column_names = ["target", "id", "date", "flag", "user", "text"]
file =('sentiment140/training.1600000.processed.noemoticon.csv')
data = pd.read_csv(file, encoding='ISO-8859-1', names=column_names)
data.head()

target          id                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....

#Part 1: Preproccessing



In [4]:
negative_tweets = data[data['target'] == 0]
positive_tweets = data[data['target'] == 4]

n_sample = 5000
negative_samples = negative_tweets.sample(n=n_sample, random_state=42)
positive_samples = positive_tweets.sample(n=n_sample, random_state=42)

In [5]:
# Normalization -> lower case
negative_samples['text'] = negative_samples['text'].str.lower()
positive_samples['text'] = positive_samples['text'].str.lower()

In [6]:
# Removing Stop Words
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    text = ' '.join(tokens)
    return text
negative_samples['text'] = negative_samples['text'].apply(lambda text: remove_stopwords(text))
positive_samples['text'] = positive_samples['text'].apply(lambda text: remove_stopwords(text))

In [7]:
# Normalization -> Remove Punctuation
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

negative_samples['text'] =negative_samples['text'].apply(lambda text: remove_punctuation(text))
positive_samples['text'] =positive_samples['text'].apply(lambda text: remove_punctuation(text))

In [8]:
# Tokenization
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens
negative_samples['text'] =negative_samples['text'].apply(lambda text: tokenize_text(text))
positive_samples['text'] =positive_samples['text'].apply(lambda text: tokenize_text(text))

In [9]:
from sklearn.model_selection import train_test_split
samples = pd.concat([negative_samples, positive_samples])
train_data, test_data = train_test_split(samples, test_size=0.2, random_state=42)

# Part2: Tf-Matrix

In [10]:
vocabulary = set()
for txt in train_data['text']:
  for word in txt:
    vocabulary.add(word)
vocabulary = sorted(list(vocabulary))

In [11]:
import numpy as np
TF_Matrix = np.zeros((len(train_data), len(vocabulary)),dtype=int)
counter = 0
for sample in train_data['text']:
  for word in sample:
    idx= vocabulary.index(word)
    TF_Matrix[counter,idx] +=1
  counter +=1

In [14]:
print(train_data.iloc[0].text)
TF_Matrix[0,vocabulary.index('need')]

['akynos', 'need', 'sit', 'one', 'day', 'drinks', 'convos', 'tweeter', 'bit', 'annoying']


1

In [13]:
TF_Dataframe = pd.DataFrame(TF_Matrix,columns=vocabulary)

# Part3: Tf-Idf

In [14]:
# Idf
N = len(TF_Matrix)
Idf_Matrix = np.zeros(len(vocabulary), dtype=float)
for i,word in enumerate(vocabulary):
    df_i = np.sum(TF_Dataframe[word])
    Idf_Matrix[i] = np.log10(N / df_i) if df_i else 0
Idf_Matrix

array([3.90308999, 3.90308999, 3.90308999, ..., 3.90308999, 3.90308999,
       3.90308999])

In [15]:
# Calculate TF-IDF
Tf_Idf = np.where(TF_Matrix > 0, 1 + np.log10(TF_Matrix), 0)
for sample in Tf_Idf:
  for word_index in range(len(vocabulary)):
    sample[word_index] = sample[word_index] * Idf_Matrix[word_index]

<ipython-input-15-ab8a11b1795e>:2: RuntimeWarning: divide by zero encountered in log10
  Tf_Idf = np.where(TF_Matrix > 0, 1 + np.log10(TF_Matrix), 0)


In [16]:
#TF-IDF of Word 'need' in Doc 0
Tf_Idf[0,vocabulary.index('need')]

1.6382721639824072

#Part4: PPMI

In [18]:
# Co-Occur Matrix
co_occurrence_matrix = np.zeros((len(vocabulary), len(vocabulary)), dtype=int)
vocab_index = {word: i for i, word in enumerate(vocabulary)}
for sample in train_data['text']:
    for i, word1 in enumerate(sample):
        for j, word2 in enumerate(sample):
            if i != j:
                index_word1 = vocab_index[word1]
                index_word2 = vocab_index[word2]
                co_occurrence_matrix[index_word1][index_word2] += 1

In [19]:
total_occurrences = co_occurrence_matrix.sum()
word_probabilities = co_occurrence_matrix.sum(axis=1) / total_occurrences
joint_probabilities = co_occurrence_matrix / total_occurrences

In [20]:
ppmi_matrix = np.zeros_like(joint_probabilities, dtype=float)
for i in range(len(vocabulary)):
    for j in range(len(vocabulary)):
        if co_occurrence_matrix[i][j] > 0:
            pmi = np.log2(joint_probabilities[i][j] / (word_probabilities[i] * word_probabilities[j]))
            ppmi_matrix[i][j] = max(pmi, 0)

In [21]:
num_samples = len(train_data['text'])
num_words = len(vocabulary)
PPMI_Vectors = np.zeros((num_samples, num_words), dtype=np.float64)

for sample_idx, sample in enumerate(train_data['text']):
    if sample:
        for word in sample:
            if word in vocab_index:
                word_idx = vocab_index[word]
                PPMI_Vectors[sample_idx] += ppmi_matrix[word_idx]

        PPMI_Vectors[sample_idx] /= len(sample)

In [22]:
PPMI_Vectors[0,vocabulary.index('need')]

2.355098565230598

#Part5: NB Classifier

## Vector of Test data based on Train Matrix

In [23]:
TF_Matrix_Test = np.zeros((len(test_data), len(vocabulary)), dtype=int)
counter = 0
for sample in test_data['text']:
    for word in sample:
        if word in vocabulary:
            idx = vocabulary.index(word)
            TF_Matrix_Test[counter, idx] += 1
    counter += 1
Tf_Idf_Test = np.where(TF_Matrix_Test > 0, 1 + np.log10(TF_Matrix_Test), 0)
for sample in Tf_Idf_Test:
    for word_index in range(len(vocabulary)):
        sample[word_index] *= Idf_Matrix[word_index]

PPMI_Vectors_Test = np.zeros((len(test_data['text']), len(vocabulary)), dtype=np.float64)
for sample_idx, sample in enumerate(test_data['text']):
    if sample:
        for word in sample:
            if word in vocab_index:
                word_idx = vocab_index[word]
                PPMI_Vectors_Test[sample_idx] += ppmi_matrix[word_idx]

        PPMI_Vectors_Test[sample_idx] /= len(sample)

<ipython-input-23-6e713a09dbbf>:9: RuntimeWarning: divide by zero encountered in log10
  Tf_Idf_Test = np.where(TF_Matrix_Test > 0, 1 + np.log10(TF_Matrix_Test), 0)


In [24]:
from sklearn.naive_bayes import MultinomialNB
train_targets = (train_data['target'] == 4).astype(int)
test_targets = (test_data['target'] == 4).astype(int)

In [25]:
nb_ppmi = MultinomialNB()
nb_tfidf = MultinomialNB()

nb_ppmi.fit(PPMI_Vectors, train_targets)
nb_tfidf.fit(Tf_Idf, train_targets)

ppmi_predictions = nb_ppmi.predict(PPMI_Vectors_Test)
tfidf_predictions = nb_tfidf.predict(Tf_Idf_Test)


In [26]:
from sklearn.metrics import precision_recall_fscore_support
precision_ppmi, recall_ppmi, f1_ppmi, _ = precision_recall_fscore_support(test_targets, ppmi_predictions, average='binary')
precision_tfidf, recall_tfidf, f1_tfidf, _ = precision_recall_fscore_support(test_targets, tfidf_predictions, average='binary')

print(f"PPMI: Precision: {precision_ppmi}, Recall: {recall_ppmi}, F1-Score: {f1_ppmi}")
print(f"TF-IDF: Precision: {precision_tfidf}, Recall: {recall_tfidf}, F1-Score: {f1_tfidf}")


PPMI: Precision: 0.6835978835978836, Recall: 0.6538461538461539, F1-Score: 0.6683911019141232
TF-IDF: Precision: 0.6765714285714286, Recall: 0.5991902834008097, F1-Score: 0.6355340848094472
